In [0]:
import requests
from pyspark.sql.functions import lit, current_timestamp, col
import json

# === API Setup ===
API_KEY = "zuNatlvzr6ZwlKh4Y9j2nQG6spfv1x2X"
LOCATION_KEY = "300597"  # Singapore

url = f"http://dataservice.accuweather.com/forecasts/v1/daily/5day/{LOCATION_KEY}?apikey={API_KEY}&metric=true"
response = requests.get(url)
forecast_data = response.json()

# === Extract DailyForecasts only ===
daily_forecasts = forecast_data.get("DailyForecasts", [])
df_weather = spark.createDataFrame(daily_forecasts)

# === Flatten the structure ===
df_flat = df_weather.select(
    col("Date"),
    col("EpochDate"),
    col("Temperature.Minimum.Value").alias("MinTemp_C"),
    col("Temperature.Maximum.Value").alias("MaxTemp_C"),
    col("Day.IconPhrase").alias("DayForecast"),
    col("Night.IconPhrase").alias("NightForecast"),
    col("Sources"),
    col("MobileLink"),
    col("Link")
).withColumn("source", lit("accuweather")) \
 .withColumn("ingestion_timestamp", current_timestamp())

# === Save as a managed Delta table under your Unity Catalog ===
df_flat.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("civAI.raw.accuweather_forecast")

# === Confirm it was saved and inspect ===
display(spark.sql("SELECT * FROM civAI.raw.accuweather_forecast"))
